In [37]:
from astropy.table import Table
import pandas as pd

import SciServer
from SciServer import Authentication, LoginPortal, Config, CasJobs, SkyQuery, SciDrive, SkyServer, Files, Jobs

Log-in

In [4]:
# Define login Name and password before running these examples
Authentication_loginName = 'name';
Authentication_loginPassword = 'Name_mark_year'

In [5]:
#logging in and getting current token from different ways

token1 = Authentication.login(Authentication_loginName, Authentication_loginPassword);
token2 = Authentication.getToken()
token3 = Authentication.getKeystoneToken()
token4 = Authentication.token.value
print("token1=" + token1)#
print("token2=" + token2)#
print("token3=" + token3)#
print("token4=" + token4)#

token1=beb10d911d90406b9bf7dfae163f63de
token2=beb10d911d90406b9bf7dfae163f63de
token3=beb10d911d90406b9bf7dfae163f63de
token4=beb10d911d90406b9bf7dfae163f63de


/home/ehko/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using SciServer.Authentication.getKeystoneToken is deprecated. Use SciServer.Authentication.getToken instead.
  """


In [6]:
#getting curent user info

user = Authentication.getKeystoneUserWithToken(token1)
print("userName=" + user.userName)
print("id=" + user.id)
iden = Authentication.identArgIdentifier()
print("ident="+iden)

userName=eunhee
id=55fcee1a408844f594223faba3040284
ident=--ident=


## CasJobs

SELECT * 
FROM specPhotoAll AS spa
  JOIN mydb.MyTable AS my ON spa.plate = my.plate_my and spa.fiberID = my.fiberid_my and spa.mjd = my.mjd_my

### upload

In [85]:
TableName = "plate_10000"
TableCSV = pd.read_csv('plate_10000.csv', index_col=None)

# print('Simple CSV file created')

In [86]:
TableCSV[:100]

,mjd_my,fiberid_my
0,57346,865
1,57346,2
2,57346,866
3,57346,5
4,57346,876
...,...,...
95,57716,244
96,57716,250
97,57716,255
98,57716,260


In [90]:
help(CasJobs.uploadPandasDataFrameToTable)

Help on function uploadPandasDataFrameToTable in module SciServer.CasJobs:

uploadPandasDataFrameToTable(dataFrame, tableName, context='MyDB')
    Uploads a pandas dataframe object into a CasJobs table. If the dataframe contains a named index, then the index will be uploaded as a column as well.
    
    :param dataFrame: Pandas data frame containg the data (pandas.core.frame.DataFrame)
    :param tableName: name of CasJobs table to be created.
    :param context: database context (string)
    :return: Returns True if the dataframe was uploaded successfully.
    :raises: Throws an exception if the user is not logged into SciServer (use Authentication.login for that purpose). Throws an exception if the HTTP request to the CasJobs API returns an error.
    :example: response = CasJobs.uploadPandasDataFrameToTable(CasJobs.getPandasDataFrameFromQuery("select 1 as foo", context="MyDB"), "NewTableFromDataFrame")
    
    .. seealso:: CasJobs.uploadCSVDataToTable



In [93]:
result =  CasJobs.uploadPandasDataFrameToTable(dataFrame=TableCSV[:100], tableName=TableName, context="MyDB")
# result = CasJobs.uploadCSVDataToTable(csvData = TableCSV, tableName=TableName, context="MyDB")


Exception: Error when uploading CSV data into CasJobs table plate_10000.
Http Response from CasJobs API returned status code 500:
{"Error Code":500,"Error Type":"InternalServerError","Error Message":"Failed to upload table data: There is not enough space on the disk.\r\n","LogMessageID":"30a0d818-7198-4bb1-8bd0-5c603dda6e29"}

In [61]:
CasJobs.executeQuery(sql="DROP TABLE " + 'temp', context="MyDB", format="pandas")

,Rows Affected
0,0


In [78]:
# df

In [100]:
myquery = 'SELECT TOP 10 * ' # note the space at the end of this string - important
myquery += 'FROM specPhotoAll AS spa '
myquery += 'JOIN mydb.plate_10000 AS my '
myquery += 'ON spa.plate = 10000 AND spa.fiberID = my.fiberid_my AND spa.mjd = my.mjd_my'

In [101]:
df = CasJobs.executeQuery(sql=myquery, context="DR17")

In [102]:
df

,specObjID,mjd,plate,tile,fiberID,z,zErr,class,subClass,zWarning,...,extinction_i,extinction_z,fieldID,dered_u,dered_g,dered_r,dered_i,dered_z,mjd_my,fiberid_my
0,11258999741428815872,57346,10000,16204,2,0.646960,0.000234,QSO,,0,...,0.115770,0.086111,1237680250024493056,21.71639,21.46008,21.00104,20.65601,20.62298,57346,2
1,11259000566062536704,57346,10000,16204,5,0.000119,0.000236,STAR,M5III (221615),0,...,0.098603,0.073343,1237680284389605376,25.93307,23.57757,22.31888,20.52351,19.82199,57346,5
2,11259000572270106624,57716,10000,16204,5,0.681351,0.000269,GALAXY,,0,...,0.113744,0.084604,1237680250024493056,22.64131,22.77679,21.68160,20.58113,19.82677,57716,5
3,11259000847148013568,57716,10000,16204,6,1.855400,0.001056,QSO,BROADLINE,0,...,0.105551,0.078511,1237680250024427520,22.20093,21.29282,21.39495,20.73722,21.02370,57716,6
4,11259001390696257536,57346,10000,16204,8,1.976910,0.000553,QSO,BROADLINE,0,...,0.095070,0.070715,1237680250561363968,21.22387,21.12267,20.87382,20.57976,20.48257,57346,8
5,11259001940452071424,57346,10000,16204,10,1.499091,0.001352,QSO,,0,...,0.090988,0.067678,1237680250561232896,22.10217,21.84498,21.62236,21.41016,22.65455,57346,10
6,11259001946659641344,57716,10000,16204,10,1.795390,0.000327,QSO,BROADLINE,0,...,0.099623,0.074101,1237680284389605376,21.12801,20.83104,20.62475,20.38811,20.25740,57716,10
7,11259002765085792256,57346,10000,16204,13,-0.000269,0.000096,STAR,F8V (30562),0,...,0.090508,0.067321,1237680250561298432,22.06158,21.53879,21.27526,21.29445,21.54703,57346,13
8,11259003595927083008,57716,10000,16204,16,0.976595,0.000134,QSO,BROADLINE,0,...,0.093845,0.069803,1237680284926476288,21.92573,21.56183,21.29002,21.35164,21.07858,57716,16
9,11259004414353233920,57346,10000,16204,19,2.128043,0.001238,QSO,BROADLINE,0,...,0.099603,0.074087,1237680284926410752,21.69278,21.48468,21.51502,21.48631,21.00229,57346,19
